In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings; warnings.filterwarnings('ignore')
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

import matplotlib.pyplot as plt
# from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, auc, precision_score, recall_score, f1_score
from sklearn import metrics

from pylab import rcParams
rcParams['figure.figsize'] = 20,5

In [1]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 32.0 MB/s 


In [3]:
!gdown --id 1diztKyKFfhINpY-8R_KB84kV8akW2NjD
!unzip -qq test_train_val.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1diztKyKFfhINpY-8R_KB84kV8akW2NjD
To: /content/test_train_val.zip
100% 480M/480M [00:01<00:00, 297MB/s]


In [4]:
with open('test_train_val/train_meta.csv') as f:
  train_df= pd.read_csv(f)
with open('test_train_val/train_signal.csv') as f:
  train_signal = pd.read_csv(f)

with open('test_train_val/valid_meta.csv') as f:
  valid_df= pd.read_csv(f)
with open('test_train_val/valid_signal.csv') as f:
  valid_signal = pd.read_csv(f)

with open('test_train_val/test_meta.csv') as f:
  test_df= pd.read_csv(f)
with open('test_train_val/test_signal.csv') as f:
  test_signal = pd.read_csv(f)

print(train_df.shape)

train_df

(17441, 37)


,ecg_id,age,sex,height,weight,nurse,site,device,NORM,MI,...,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI,strat_fold
0,1,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
1,2,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,2
2,3,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,5
3,4,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
4,5,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,21832,63.0,0,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17437,21833,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17438,21834,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,1,0,...,0,0,0,0,0,0,0,0,0,4
17439,21835,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,2


In [5]:
class PTBXLDatasetPreprocesser():
    def __init__(self):
        pass
    
    def save(self, filename):
        data = {
            'superclass_cols': self.superclass_cols,
            'subclass_cols': self.subclass_cols,
            'meta_num_cols': self.meta_num_cols,
            'meta_num_means': self.meta_num_means,
            'min_max_scaler': self.min_max_scaler,
            'meta_cat_cols': self.meta_cat_cols,
            'cat_lablers': self.cat_lablers,
            'bclass_cols': self.bclass_cols,
            
        }
        pd.to_pickle(data, filename)
        
    def load(self, filename):
        data = pd.read_pickle(filename)
        self.min_max_scaler = data['min_max_scaler']
        self.cat_lablers = data['cat_lablers']
        #self.binary_lablers = data['binary_lablers']
        
    def fit(self, x, y):
        x = x.copy()
        y = y.copy()
        
        self.superclass_cols = [ 'MI', 'STTC', 'CD', 'HYP']
        
        self.subclass_cols = [col for col in y.columns if 'sub_' in col]

        self.bclass_cols = ['NORM']
        
        self.meta_num_cols = ['age', 'height', 'weight']
        self.meta_num_means = []
        for col in self.meta_num_cols:
            print(col, y[col].mean())
            y[col] = y[col].fillna(y[col].mean())
            self.meta_num_means += [y[col].mean()]
            
        self.min_max_scaler = MinMaxScaler().fit(y[self.meta_num_cols])
        
        self.meta_cat_cols = ['sex'] #, 'nurse', 'device']
        self.cat_lablers = [LabelEncoder().fit(y[col].fillna('none').astype(str)) for col in self.meta_cat_cols]
        return self

        #self.min_max_scaler = MinMaxScaler().fit(y[self.meta_num_cols])
        
        #self.meta_binary_cols = ['NORM'] 
        #self.binary_lablers = [LabelEncoder().fit(y[col].fillna('none').astype(str)) for col in self.meta_binary_cols]
        #return self
    
    def transform(self, x, y):
        
        channel_cols = x.columns.tolist()[1:]
        
        ret = []
        x = x[channel_cols].values.reshape(-1, 1000, 12)
        print(x.shape)
        ret += [x] # signal
        
        y_ = y.copy()
        
        for i, col in enumerate(self.meta_num_cols):
            y_[col] = y_[col].fillna(self.meta_num_means[i])
        y_[self.meta_num_cols] = self.min_max_scaler.transform(y_[self.meta_num_cols])
        y_[self.meta_num_cols] = np.clip(y_[self.meta_num_cols], 0., 1.) # prevent extreme value far from train set
        
        ret += [y_[self.meta_num_cols]] # meta num features
        
        for i, col in enumerate(self.meta_cat_cols):
            y_[col] = y_[col].fillna('none').astype(str)
            y_[col] = self.cat_lablers[i].transform(y_[col]) 
        
        ret += [y_[self.meta_cat_cols]] # meta cat features

        #for i, col in enumerate(self.meta_binary_cols):
           # y_[col] = y_[col].fillna('none').astype(str)
           # y_[col] = self.binary_lablers[i].transform(y_[col]) 
        
        #ret += [y_[self.meta_binary_cols]] # binary class target
        
        if np.isin(self.superclass_cols, y.columns).sum() == len(self.superclass_cols):
            ret += [y[self.superclass_cols].fillna(0).astype(int)] # superclass targets
        
        if np.isin(self.subclass_cols, y.columns).sum() == len(self.subclass_cols):
            ret += [y[self.subclass_cols].fillna(0).astype(int)] # subclass targets

        if np.isin(self.bclass_cols, y.columns).sum() == len(self.bclass_cols):
            ret += [y[self.bclass_cols].fillna(0).astype(int)]
        
        return ret

In [6]:
data_preprocessor = PTBXLDatasetPreprocesser()
data_preprocessor.fit(train_signal, train_df)
train_signal, train_meta_num_feats, train_meta_cat_feats, train_superclass, train_subclass, train_bclass = data_preprocessor.transform(train_signal, train_df)
valid_signal, valid_meta_num_feats, valid_meta_cat_feats, valid_superclass, valid_subclass, valid_bclass = data_preprocessor.transform(valid_signal, valid_df)
test_signal, test_meta_num_feats, test_meta_cat_feats, test_superclass, test_subclass, test_bclass = data_preprocessor.transform(test_signal, test_df)

print(train_signal.shape)
print(valid_meta_num_feats.isna().sum(), valid_meta_cat_feats.isna().sum(), valid_superclass.isna().sum(), valid_subclass.isna().sum(), valid_bclass.isna().sum())

display(train_meta_num_feats)
display(train_meta_cat_feats)
display(train_superclass)
display(train_subclass)
display(train_bclass)

age 59.63709074169975
height 166.68908469699693
weight 70.69599447513812
(17441, 1000, 12)
(2193, 1000, 12)
(2203, 1000, 12)
(17441, 1000, 12)
age       0
height    0
weight    0
dtype: int64 sex    0
dtype: int64 MI      0
STTC    0
CD      0
HYP     0
dtype: int64 sub_NORM         0
sub_IMI          0
sub_STTC         0
sub_NST_         0
sub_LVH          0
sub_LAFB/LPFB    0
sub_RVH          0
sub_RAO/RAE      0
sub_IRBBB        0
sub_IVCD         0
sub_LMI          0
sub_AMI          0
sub__AVB         0
sub_ISCA         0
sub_ISC_         0
sub_SEHYP        0
sub_ISCI         0
sub_CRBBB        0
sub_CLBBB        0
sub_LAO/LAE      0
sub_ILBBB        0
sub_WPW          0
sub_PMI          0
dtype: int64 NORM    0
dtype: int64


,age,height,weight
0,0.580645,0.791572,0.282927
1,0.182796,0.791572,0.317073
2,0.376344,0.791572,0.312195
3,0.236559,0.791572,0.375610
4,0.182796,0.791572,0.317073
...,...,...,...
17436,0.655914,0.791572,0.320468
17437,0.698925,0.791572,0.320468
17438,0.978495,0.791572,0.320468
17439,0.612903,0.791572,0.320468


,sex
0,1
1,0
2,1
3,0
4,1
...,...
17436,0
17437,1
17438,0
17439,1


,MI,STTC,CD,HYP
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
17436,0,0,1,0
17437,0,1,0,0
17438,0,0,0,0
17439,0,1,0,0


,sub_NORM,sub_IMI,sub_STTC,sub_NST_,sub_LVH,sub_LAFB/LPFB,sub_RVH,sub_RAO/RAE,sub_IRBBB,sub_IVCD,...,sub_ISCA,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17437,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17438,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17439,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


,NORM
0,1
1,1
2,1
3,1
4,1
...,...
17436,0
17437,0
17438,1
17439,0


In [7]:
signal=np.array(train_signal)
target=np.array(train_bclass)

v_signal=np.array(valid_signal)
v_target=np.array(valid_bclass)

t_signal=np.array(test_signal)
t_target=np.array(test_bclass)

In [8]:
full_signal= np.vstack((signal,v_signal,t_signal))
full_signal.shape

(21837, 1000, 12)

In [9]:
full_target= np.vstack((target,v_target,t_target))
full_target.shape

(21837, 1)

In [10]:
new_signal = full_signal.reshape(21837,12000)

In [11]:
x1 = new_signal[:,0:2400]
x2 = new_signal[:,2400:4800]
x3 = new_signal[:,4800:7200]
x4 = new_signal[:,7200:9600]
x5 = new_signal[:,9600:12000]

y = full_target[:,0]

In [14]:
classifier = []

In [13]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, X4_train, X4_test, X5_train, X5_test, Y_train, Y_test = train_test_split(x1,x2,x3,x4,x5, y, test_size=0.15, random_state=25)

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [16]:
model = SVC(kernel='rbf')
clf1 = model.fit(X1_train, Y_train)

In [17]:
# pred_labels_tr = model.predict(X1_train)
pred_labels_te = model.predict(X1_test)
classifier.append(np.array(pred_labels_te))

In [18]:
# score_tr = model.score(X1_train, Y_train)
score_te = model.score(X1_test, Y_test)

# print('Accuracy svm kernel=rbf train: ', score_tr)
print('Accuracy svm kernel=rbf test: ', score_te)

Accuracy svm kernel=rbf test:  0.7396214896214897


In [ ]:
#  print(classification_report(Y_train, pred_labels_tr))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89     10472
           1       0.86      0.86      0.86      8089

    accuracy                           0.88     18561
   macro avg       0.88      0.88      0.88     18561
weighted avg       0.88      0.88      0.88     18561



In [ ]:
 print(classification_report(Y_test, pred_labels_te))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77      1837
           1       0.71      0.70      0.70      1439

    accuracy                           0.74      3276
   macro avg       0.74      0.74      0.74      3276
weighted avg       0.74      0.74      0.74      3276



In [ ]:
# X2_train, X2_test, Y_train, Y_test = train_test_split(x2, y, test_size=0.15, random_state=25)

In [19]:
model = SVC(kernel='rbf')
clf2 = model.fit(X2_train, Y_train)

In [20]:
# pred_labels_tr = model.predict(X2_train)
pred_labels_te = model.predict(X2_test)
classifier.append(np.array(pred_labels_te))

In [21]:
# score_tr = model.score(X2_train, Y_train)
score_te = model.score(X2_test, Y_test)

# print('Accuracy svm kernel=rbf train: ', score_tr)
print('Accuracy svm kernel=rbf test: ', score_te)

Accuracy svm kernel=rbf test:  0.7777777777777778


In [ ]:
# X3_train, X3_test, Y_train, Y_test = train_test_split(x3, y, test_size=0.15, random_state=25)

In [22]:
model = SVC(kernel='rbf')
clf3 = model.fit(X3_train, Y_train)

In [23]:
# pred_labels_tr = model.predict(X3_train)
pred_labels_te = model.predict(X3_test)
classifier.append(np.array(pred_labels_te))

In [24]:
# score_tr = model.score(X3_train, Y_train)
score_te = model.score(X3_test, Y_test)

# print('Accuracy svm kernel=rbf train: ', score_tr)
print('Accuracy svm kernel=rbf test: ', score_te)

Accuracy svm kernel=rbf test:  0.7780830280830281


In [ ]:
# X4_train, X4_test, Y_train, Y_test = train_test_split(x4, y, test_size=0.15, random_state=25)

In [25]:
model = SVC(kernel='rbf')
clf = model.fit(X4_train, Y_train)

In [26]:
# pred_labels_tr = model.predict(X4_train)
pred_labels_te = model.predict(X4_test)
classifier.append(np.array(pred_labels_te))

In [27]:
# score_tr = model.score(X4_train, Y_train)
score_te = model.score(X4_test, Y_test)

# print('Accuracy svm kernel=rbf train: ', score_tr)
print('Accuracy svm kernel=rbf test: ', score_te)

Accuracy svm kernel=rbf test:  0.7774725274725275


In [ ]:
# X5_train, X5_test, Y_train, Y_test = train_test_split(x5, y, test_size=0.15, random_state=25)

In [28]:
model = SVC(kernel='rbf')
clf5 = model.fit(X5_train, Y_train)


In [29]:
# pred_labels_tr = model.predict(X5_train)
pred_labels_te = model.predict(X5_test)
classifier.append(np.array(pred_labels_te))

In [30]:
# score_tr = model.score(X5_train, Y_train)
score_te = model.score(X5_test, Y_test)

# print('Accuracy svm kernel=rbf train: ', score_tr)
print('Accuracy svm kernel=rbf test: ', score_te)

Accuracy svm kernel=rbf test:  0.7649572649572649


In [31]:
y_prob = np.array(classifier).transpose()

In [32]:
y_prob.shape

(3276, 5)

In [33]:
import collections

In [34]:
final_out=[]
for i in range(0, y_prob.shape[0]):
  a=np.array(y_prob[i,:])
  a= collections.Counter(a).most_common()[0][0]
  final_out.append(a)

In [35]:
accuracy_score(Y_test,final_out)

0.7982295482295483